In [74]:
import sqlite3 as sq
import numpy as np

from IPython.display import HTML, display
import tabulate

In [11]:
conn = sq.connect('../food.db')
c = conn.cursor()

In [3]:
c.execute('select DISTINCT countries from food_facts where origins LIKE \'%au%\';')
c.fetchone()

('Royaume-Uni,France',)

In [3]:
# We'll add this data to a seperate table for easier analysis
c.execute('DROP TABLE IF EXISTS food_origins;')
c.execute('\
        CREATE TABLE food_origins (\
             code varchar not null,\
             country varchar not null\
        );')

c.execute('select code, countries_en from food_facts;')
for row in c.fetchall():
    if row[1] == '': continue
    for country in row[1].split(','):
        c.execute('INSERT INTO food_origins (code, country) VALUES (?,?);', [row[0], country])


In [4]:
c.execute('CREATE INDEX food_origin_index ON food_origins(code, country)')

In [24]:
c.execute('SELECT DISTINCT country from food_origins where country = \'AU\'')
c.fetchall()

[]

In [7]:
c.execute('select DISTINCT manufacturing_places_tags from food_facts where manufacturing_places_tags like \'%aus%\'')
c.fetchone()

('australia',)

In [7]:
# The text 'austral' seems to cover all of Australian origins, including German spellings of Aus
c.execute('SELECT * FROM food_facts WHERE manufacturing_places_tags LIKE \'%austral%\'')
c.fetchone()

('0062020000248',
 'http://world-en.openfoodfacts.org/product/0062020000248/nutella-ferrero',
 'foodorigins',
 '1447895880',
 '2015-11-19T01:18:00Z',
 '1451885453',
 '2016-01-04T05:30:53Z',
 'Nutella',
 '',
 '750g',
 'Plastic Jar',
 'plastic-jar',
 'Ferrero',
 'ferrero',
 'Chocolate Spread',
 'en:spreads,en:breakfasts,en:sweet-spreads,fr:pates-a-tartiner,en:chocolate-spreads',
 'Spreads,Breakfasts,Sweet spreads,fr:Pâtes à tartiner,Chocolate spreads',
 'Australia',
 'australia',
 'Australia',
 'australia',
 'Australian Made',
 'en:australian-made',
 'Australian-made',
 '',
 '',
 '',
 '',
 '',
 'NSW,Australia',
 'Coles,Bilo',
 'Australia',
 'en:australia',
 'Australia',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Sugary snacks',
 'Sweets',
 'en:to-be-completed, en:nutrition-facts-to-be-completed, en:ingredients-to-be-completed, en:expiration-date-completed, en:characteristics-completed, en:categories-completed, en:brands-completed

In [10]:
# Now where are the peanut butters?
c.execute('SELECT DISTINCT categories_tags from food_facts where categories_tags LIKE \'%peanut%\'')
c.fetchone()

('en:plant-based-foods-and-beverages,en:plant-based-foods,en:spreads,en:legumes-and-their-products,en:plant-based-spreads,en:nuts-and-their-products,en:oilseed-purees,en:nut-butters,en:legume-butters,en:peanut-butters,en:crunchy-peanut-butter',)

In [50]:
# Looks like this column must be processed too...
# We'll add this data to a seperate table for easier analysis
c.execute('DROP TABLE IF EXISTS food_category;')
c.execute('\
        CREATE TABLE food_category (\
             code varchar not null,\
             cat varchar not null\
        );')

c.execute('select code, categories_tags from food_facts;')
for row in c.fetchall():
    if row[1] == '': continue
    for cat in row[1].split(','):
        c.execute('INSERT INTO food_category (code, cat) VALUES (?,?);', [row[0], cat])

In [9]:
# Determine which categories we want to include
c.execute('SELECT DISTINCT cat from food_category where cat LIKE \'%peanut%\'')
c.fetchall()

[('en:peanut-butters',),
 ('en:crunchy-peanut-butter',),
 ('en:peanuts',),
 ('en:peanut-butter-bliss',),
 ('en:roasted-peanuts',),
 ('en:unsalted-dry-roasted-peanuts',),
 ('en:dry-roasted-peanuts',),
 ('en:peanut-butter-cup',),
 ('en:creamy-peanut-butter',),
 ('en:smooth-peanut-butter',),
 ('en:chocolate-peanut-butter',),
 ('en:chocolate-covered-peanuts',),
 ('en:peanut-sauce',),
 ('en:peanut-butter-and-chocolat-chip-cereal-bar',),
 ('en:peanut-bars',),
 ('en:peanut-oils',),
 ('en:caramelized-peanuts',),
 ('en:shelled-peanuts',),
 ('en:roasted-shelled-peanuts',),
 ('en:unshelled-peanuts',),
 ('en:peanut-in-chocolate',),
 ('en:peanuts-turron',),
 ('en:peanut-butter-blended-with-rich-dark-chocolate',),
 ('en:salted-peanuts',),
 ('en:unsalted-peanuts',)]

In [10]:
# We want all peanut butter type except the peanut butter cup, which I think is a sweet, and peanut butter bar
c.execute('\
    select code from food_category \
    where cat LIKE \'%peanut-butter%\'\
    and cat != \'en:peanut-butter-cup\' AND cat !=\'en:peanut-butter-and-chocolat-chip-cereal-bar\' \
')
c.fetchone()

('0011110016508',)

In [75]:
c.execute('\
    SELECT quantity, product_name, generic_name, creator, countries_en, manufacturing_places FROM food_facts\
    WHERE code IN (\
        SELECT code FROM food_origins WHERE country LIKE \'%austral%\' OR country = \'en:AU\'\
    )\
    AND code IN (\
        select code from food_category \
        where cat LIKE \'%peanut-butter%\'\
        and cat != \'en:peanut-butter-cup\' AND cat != \'en:peanut-butter-and-chocolat-chip-cereal-bar\' \
    )')
heading = ('Quantity', 'Product Name', 'Generic Name', 'Creator', 'Countries', 'Place of Manufacture')
display(HTML(tabulate.tabulate(c.fetchall(), heading, tablefmt='html')))

Quantity,Product Name,Generic Name,Creator,Countries,Place of Manufacture
739 g,Bramwells American Style Peanut Butter,"Peanut Butter, Smooth",openfoodfacts-contributors,Australia,United States
500g,Bramwells Peanut Butter Crunchy,,foodorigins,Australia,Argentina
375g,Crunchy Peanut Butter,,foodorigins,Australia,China
375g,Smooth Peanut Butter,,foodorigins,Australia,India
500g,Smooth Peanut Butter,,foodorigins,Australia,China
500g,Crunchy Peanut Butter,,foodorigins,Australia,China
780g,Peanut Butter,,openfoodfacts-contributors,Australia,
,Peanut Butter,,openfoodfacts-contributors,Australia,
,Peanut butter,Peanut butter,foodorigins,Australia,
375 g,natural peanut butter crunchy,,svendowideit,Australia,


In [5]:
# These countries appear the below number of times in the country field
c.execute('\
    EXPLAIN QUERY PLAN SELECT country, count(country),\
          (\
              SELECT count(*) from food_facts\
              WHERE code IN (\
                  SELECT code FROM food_origins\
                  WHERE country = fo1.country\
              )\
              AND zinc_100g >0.001\
          ) AS `products with zinc above threshold`\
      FROM food_origins fo1 \
      GROUP BY country\
      ORDER BY count(country)\
      DESC LIMIT 10')
c.fetchall()

[(0, 0, 0, 'SCAN TABLE food_origins AS fo1'),
 (0, 0, 0, 'USE TEMP B-TREE FOR GROUP BY'),
 (0, 0, 0, 'EXECUTE CORRELATED SCALAR SUBQUERY 1'),
 (1, 0, 0, 'SCAN TABLE food_facts'),
 (1, 0, 0, 'EXECUTE CORRELATED LIST SUBQUERY 2'),
 (2, 0, 0, 'SCAN TABLE food_origins'),
 (0, 0, 0, 'USE TEMP B-TREE FOR ORDER BY')]

In [76]:
c.execute('\
    SELECT country, count(country) FROM food_origins\
    GROUP BY country \
    ORDER BY count(country) \
    DESC LIMIT 10\
')
display(HTML(tabulate.tabulate(c.fetchall(), ('Country', 'Quantity'), tablefmt='html')))

Country,Quantity
United States,173547
France,109734
Switzerland,17053
Germany,8932
United Kingdom,5781
Spain,5750
Belgium,3669
Australia,2268
Russia,1517
Italy,1435


In [27]:
c.execute('''SELECT zinc_100g FROM food_facts
          WHERE zinc_100g > 0.001;''')
c.fetchone()

('0.01053',)

In [77]:
# I found this one quite complex. I took it to mean the top 10 countries by number
# of products that have its name under 'countries', then sort by and display number of products
# for each of these 10 countries that have zinc above 0.001
c.execute('''
SELECT country, count(country) AS `products with zinc above threshold` from food_origins fo
    WHERE country IN (
        SELECT country FROM food_origins
        GROUP BY country
        ORDER BY count(country)
        DESC LIMIT 10
    )
    AND code IN (
          SELECT code FROM food_facts
          WHERE zinc_100g > 0.001
    )
    GROUP BY country
    ORDER BY count(country) DESC;
''')
display(HTML(tabulate.tabulate(c.fetchall(), ('Country', 'products with zinc above threshold'), tablefmt='html')))

Country,products with zinc above threshold
United States,2802
France,174
Switzerland,65
Germany,42
Spain,13
Belgium,10
United Kingdom,6
Russia,5
Australia,4
Italy,3


In [36]:
c.execute('''
    SELECT DISTINCT ingredients_text FROM food_facts
    WHERE ingredients_text LIKE '%chicken%'
''')
c.fetchone()

('White chicken, chicken broth, salt.',)

In [46]:
# We'll add this data to a seperate table for easier analysis
c.execute('DROP TABLE IF EXISTS food_ingredients;')
c.execute('\
        CREATE TABLE food_ingredients (\
             code varchar not null,\
             ingredient varchar not null\
        );')

c.execute('select code, ingredients_text from food_facts;')
for row in c.fetchall():
    if row[1] == '': continue
    for cat in row[1].split(','):
        c.execute('INSERT INTO food_ingredients (code, ingredient) VALUES (?,?);', [row[0], cat])

In [47]:
c.execute('CREATE INDEX food_ingredients_index ON food_ingredients(code, ingredient)')

In [78]:
c.execute('''
    SELECT DISTINCT ingredient FROM food_ingredients
    WHERE ingredient LIKE '%chicken%'
    OR ingredient LIKE '%pork%'
    OR ingredient LIKE '%tofu%'
''')
c.fetchone()

(' Cooked Italian Sausage (Pork',)

In [113]:
ingredients = ('chicken', 'pork', 'tofu')
protein_stats = []

for i in ingredients:


    # The last two calculations were too inconvenient to do in sqlite, so opted for numpy solution
    c.execute("""
        SELECT CAST(proteins_100g AS FLOAT)
        FROM food_facts
        WHERE code IN (
            SELECT DISTINCT code FROM food_ingredients
            WHERE ingredient LIKE '%"""+ i +"""%'

        )
        AND proteins_100g != ''
        ORDER BY proteins_100g
    """)
    protein_all = c.fetchall()
    
    avg = np.mean(protein_all)
    med = np.median(protein_all)
    
    stddev = np.std(protein_all)
    
    protein_stats.append((i, avg, med, stddev))
    
heading=('Country', 'Avg', 'Median', 'Std Dev')
display(HTML(tabulate.tabulate(protein_stats, heading, tablefmt='html')))

Country,Avg,Median,Std Dev
chicken,10.0564,9.73,7.07095
pork,16.9258,14.68,11.1602
tofu,10.8612,9.78,6.99405


In [103]:
display(HTML(tabulate.tabulate(protein_stats, tablefmt='html')))

"(10.056430954391855,)","(16.92583901773535,)","(10.861164122137417,)"
,,


In [10]:
conn.commit()
conn.close()

In [ ]:
# Combine queries
